# Exercice 1

**Question 1** - Écrire une fonctions `debut : int -> 'a list -> 'a list` qui prend en argument un entier positif $n$ et une liste $\ell$ et qui renvoie la liste des $n$ premiers éléments de $\ell$, ou la liste en entier si cette dernière possède moins de $n$ éléments.

In [1]:
let rec debut n lst =
  match n, lst with
  | 0, _ -> []
  | _, [] -> []
  | _, t::q -> t::(debut (n-1) q)
;;

val debut : int -> 'a list -> 'a list = <fun>


**Question 2** - Écrire une fonctions `iterer : int -> ('a -> 'a) -> 'a -> 'a list` qui prend en argument un entier positif $n$, une fonction $f$ de type `'a -> 'a` et un élément $x$ de type `'a` et qui renvoie la liste $[x;f(x);f^2(x);\dots;f^n(x)]$.

In [2]:
let rec iterer n f x =
  match n with
  | 0 ->  [x]
  | _ -> x::iterer (n-1) f (f x)
;;

val iterer : int -> ('a -> 'a) -> 'a -> 'a list = <fun>


**Question 3** - Écrire une fonctions `init : int -> (int -> 'a) -> 'a list` qui prend en argument un entier positif $n$, une fonction $f$ de type `'int -> 'a` et qui renvoie la liste vide si $n$ est nul, et la liste $[f(0);f(1);\dots;f(n-1)]$ si $n > 0$.

In [3]:
let init n f =
  let rec construit k =
    (* Construit la liste [f(k);...;f(n-1)]*)
    if k = n
    then []
    else f k::construit (k+1)
  in construit 0
;;

val init : int -> (int -> 'a) -> 'a list = <fun>


In [4]:
let init n f =
  let rec aux i = 
    match i with
    | 0 -> []
    | _ -> (f (n-i))::aux (i-1)
  in aux n

val init : int -> (int -> 'a) -> 'a list = <fun>


In [5]:
init 5 (fun i -> i*i)

- : int list = [0; 1; 4; 9; 16]


**Question 4** - Écrire une fonctions `somme : int -> int list -> bool` qui prend en argument un entier $n$, une liste d'entiers $\ell$ et qui renvoie `true` s'il existe une partie des éléments de $\ell$ dont la somme vaut $n$, `false` sinon.

Par exemple, l'appel `somme 10 [8; -7; 4; 9]` renverra `true` (en effet, $10 = 8+(-7)+9$) mais  l'appel `somme 11 [8; -7; 4; 9]` renverra `false`.
*On ne demande pas nécessairement une fonction efficace.*

In [6]:
let rec somme n lst =
  match lst with
  | [] -> n = 0
  | t::q -> somme (n-t) q || somme n q
;;

val somme : int -> int list -> bool = <fun>


# Exercice 2 

Le lycée Uderzo dispose de trois classes de MPSI, d'une PSI et de deux MP.\medskip

On souhaite définir trois types `sexe`, `division` et `etudiant`, de sorte que Bonemine ABRARACOURCIX, étudiante en MPSI 3, puisse être représenter par la variable `bonemine` suivante :

`let bonemine = {
    nom = "ABRARACOURCIX";
    prenom = "Bonemine";
    classe = MPSI 3; (* de type division *)
    sexe = F (* de type sexe *)
  };;     ` 

**Question 1** - Définir les types `sexe` et `division`. Le type `division` comportera trois constructeurs dont deux seront paramétrés par un entier.

In [7]:
type sexe = M | F;;

type sexe = M | F


In [8]:
type division =
  | MPSI of int
  | MP of int
  | PSI
;;

type division = MPSI of int | MP of int | PSI


**Question 2** - Définir le type `etudiant`.

In [9]:
type etudiant = {
    nom : string;
    prenom : string;
    classe : division;
    sexe : sexe
  };;

type etudiant = {
  nom : string;
  prenom : string;
  classe : division;
  sexe : sexe;
}


**Question 3** - Écrire une fonction `repartition_mp : etudiant list -> etudiant list * etudiant list` prenant en argument une liste d'étudiants `lst` et renvoyant un couple de listes `(lst1, lst2)` où `lstk` est la liste des étudiants de la MP k. La liste de départ pourra contenir des étudiants des autres classes.

In [10]:
let rec repartition_mp lst =
  match lst with
  | [] -> [], []
  | t::q -> let mp1, mp2 = repartition_mp q in
            match t.classe with
            | MP 1 -> t::mp1, mp2
            | MP 2 ->mp1, t::mp2
            | _ -> mp1, mp2
;;

val repartition_mp : etudiant list -> etudiant list * etudiant list = <fun>


**Question 4** - Écrire une fonction `repartion_sexe : etudiant list -> int * int` prenant en argument une liste d'étudiants `lst` et renvoyant le couple d'entiers $(n,p)$ où $n$ est le nombre d'étudiantes de la liste `lst`, et $p$ le nombre d'étudiants masculins.

In [11]:
let rec repartition_sexe lst =
  match lst with
  | [] -> 0, 0
  | t::q -> let nq, pq = repartition_sexe q in
            match t.sexe with
            | F -> nq + 1, pq
            | M -> nq, pq +1
;;

val repartition_sexe : etudiant list -> int * int = <fun>


**Question 5** - Écrire une fonction `proportion_filles : etudiant list -> float` prenant en argument une liste d'étudiants `lst` et renvoyant la proportion d'étudiantes dans la liste `lst`.
    

In [12]:
let proportion_filles lst =
  let n, p = repartition_sexe lst in
  float_of_int n /. (float_of_int (n+p))
;;

val proportion_filles : etudiant list -> float = <fun>


**Question 6** - On souhaite rééquilibrer les effectifs des classes de MP 1 et de MP 2, de sorte que :

- le nombre total d'étudiants diffère au plus d'une unité ;
- le nombre d'étudiantes (resp. d'étudiants masculins) diffère au plus d'une unité.

Écrire une fonction
 `nouvelle_repartition : etudiant list -> etudiant list -> etudiant list * etudiant list`
prenant en argument les deux listes correspondant aux étudiants de chacune des MP et renvoyant un couple de deux listes correspondant à la nouvelle répartition.

> **Une première méthode** : *On calcule le nombre de filles et de garçons à atteindre en MP1 grâce à la fonction repartition_sexe ; on fait le choix arbitraire d'avoir éventuellement une fille de moins et/ou un garçon de plus en MP1 (ce qui permet de garantir les trois contraintes sur les effectifs).*

In [13]:
let nouvelle_repartition lst1 lst2 =
  let n1, p1 = repartition_sexe lst1 in
  let n2, p2 = repartition_sexe lst2 in
  let new_n1, new_p1 = (n1+n2)/2, (p1+p2+1)/2 in
  let rec construit_mp1 l1 l2 n p =
    match l1, l2 with
    | [], [] -> [], []
    | t1::q1,_ ->
       begin
         match t1.sexe, n, p with
         | F, 0, _ ->  let mp1, mp2 = construit_mp1 q1 l2 0 p in mp1, t1::mp2
         | F, _, _ -> let mp1, mp2 = construit_mp1 q1 l2 (n-1) p in t1::mp1, mp2
         | M, _, 0 -> let mp1, mp2 = construit_mp1 q1 l2 n 0 in  mp1, t1::mp2
         | M, _, _ ->  let mp1, mp2 = construit_mp1 q1 l2 n (p-1) in t1::mp1, mp2
       end
    | _, t2::q2 ->
       begin
         match t2.sexe,n, p with
         | F, 0, _ ->  let mp1, mp2 = construit_mp1 l1 q2 0 p in mp1, t2::mp2
         | F, _, _ -> let mp1, mp2 = construit_mp1 l1 q2 (n-1) p in t2::mp1, mp2
         | M, _, 0 -> let mp1, mp2 = construit_mp1 l1 q2 n 0 in  mp1, t2::mp2
         | M, _, _ ->  let mp1, mp2 = construit_mp1 l1 q2 n (p-1) in t2::mp1, mp2
       end
  in construit_mp1 lst1 lst2 new_n1 new_p1
;;

val nouvelle_repartition :
  etudiant list -> etudiant list -> etudiant list * etudiant list = <fun>


> *Petite variante proposée par $*$ : on échange les deux classes lorsque la liste des mp1 restants est épuisée. Cela nécessite toutefois de garder le nombre de filles et de garçons à ajouter dans chaque classe*


In [22]:
let nouvelle_repartition lst1 lst2 =
  let n1, p1 = repartition_sexe lst1 in
  let n2, p2 = repartition_sexe lst2 in
  let new_n1, new_p1 = (n1+n2)/2, (p1+p2+1)/2 in
  let rec construit l1 l2 n1 p1 n2 p2  =
    match l1, l2 with
    | [], [] -> [], []
    | t1::q1,_ ->
       begin
         match t1.sexe, n1, p1 with
         | F, 0, _ ->  let mp1, mp2 = construit q1 l2 n1 p1  (n2-1) p2 in mp1, t1::mp2
         | F,_, _ -> let mp1, mp2 = construit q1 l2 (n1-1) p1 n2 p2 in t1::mp1, mp2
         | M, _, 0 -> let mp1, mp2 = construit q1 l2 n1 p1 n2 (p2-1) in  mp1, t1::mp2
         | M, _, _ ->  let mp1, mp2 = construit q1 l2 n1 (p1-1) n2 p2 in t1::mp1, mp2
       end
    | _, t2::q2 -> construit l2 l1 n2 p2 n1 p1
  in construit lst1 lst2 new_n1 new_p1 (n1+n2-new_n1) (p1+p2-new_p1)
;;

val nouvelle_repartition :
  etudiant list -> etudiant list -> etudiant list * etudiant list = <fun>


> **Une deuxième méthode** : *(d'après les propositions de $*$ et Flavien)*
>
> *On répartit les étudiants deux par deux en mémorisant l'écart entre le nombre de filles en mp1 et celui en mp2.*
>
> *On fait à nouveau le choix d'avoir éventuellement une fille de plus et/ou un garçon de moins en mp1*

In [34]:
let nouvelle_repartition lst1 lst2 =
  let rec construit l1 l2 new_mp1 new_mp2 delta =
    match l1, l2 with
    | [], [] ->  new_mp1 , new_mp2
    | [t1], [] ->
      if t1.sexe = F
      then
        if delta > 0
        then new_mp1, t1::new_mp2
        else t1::new_mp1, new_mp2
      else
      if delta > 0 (* plus de filles en mp1, donc moins de garçons *)
      then t1::new_mp1, new_mp2
      else new_mp1, t1::new_mp2
    | t1::t2::q1, [] -> construit (t1::q1) [t2] new_mp1 new_mp2 delta
    | [], _ -> construit l2 [] new_mp1 new_mp2 delta
    | t1::q1,t2::q2 ->
      if t1.sexe = t2.sexe
      then construit q1 q2 (t1::new_mp1) (t2::new_mp2) delta
      else
        match t1.sexe, delta > 0 with
        | F, true -> construit q1 q2 (t2::new_mp1) (t1::new_mp2) (delta - 1)
        | F, false -> construit q1 q2 (t1::new_mp1) (t2::new_mp2) (delta + 1)
        | M, true -> construit q1 q2 (t1::new_mp1) (t2::new_mp2) delta
        | M, false -> construit q1 q2 (t2::new_mp1) (t1::new_mp2) delta
  in construit lst1 lst2 [] [] 0
;;

val nouvelle_repartition :
  etudiant list -> etudiant list -> etudiant list * etudiant list = <fun>


> **Pour terminer** : *(d'après les propositions de $*$, Maxence et Camille G.)*
>
> On calcule la liste de toutes les filles de MP et la liste de tous les garçons, puis on les répartit.

In [31]:
let nouvelle_repartition lst1 lst2 =
  let rec listes_sexe lstTous lstF lstM =
    match lstTous with
    | [] -> lstF, lstM
    | t::q -> 
      if t.sexe = F 
      then listes_sexe q (t::lstF) lstM
      else listes_sexe q lstF (t::lstM)
    in
    let rec repartir lst (mp1,mp2) =
      match lst with
      | [] -> mp1, mp2
      | [t1] -> t1::mp1, mp2
      | t1::t2::q -> repartir q (t1::mp1,t2::mp2)
    in
    let lstF, lstM = listes_sexe (lst1@lst2) [] [] in
    repartir lstF (repartir lstM ([],[]))
;;

val nouvelle_repartition :
  etudiant list -> etudiant list -> etudiant list * etudiant list = <fun>


> *Variante proposée par Cyril, sans regrouper les classes*

In [33]:
let nouvelle_repartition lst1 lst2 =
  let rec listes_sexe lstTous lstF lstM =
    match lstTous with
    | [] -> lstF, lstM
    | t::q -> 
      if t.sexe = F 
      then listes_sexe q (t::lstF) lstM
      else listes_sexe q lstF (t::lstM)
    in
    let rec repartir lst (mp1,mp2) =
      match lst with
      | [] -> mp1, mp2
      | [t1] -> t1::mp1, mp2
      | t1::t2::q -> repartir q (t1::mp1,t2::mp2)
    in
    let lstF1, lstM1 = listes_sexe lst1 [] [] in
    let lstF2, lstM2 = listes_sexe lst2 [] [] in
    let new_mp1F, new_mp2F = repartir lstF1 (repartir lstF2 ([],[])) in
    repartir lstM1 (repartir lstM2 (new_mp1F, new_mp2F))
;;

val nouvelle_repartition :
  etudiant list -> etudiant list -> etudiant list * etudiant list = <fun>


# Exercice 3

On étudie ici la fonction `cantor` définie par
$$\forall (x,y) \in \mathbb{N}^2,\, \mathtt{cantor}(x,y) = x + \frac{(x+y)(x+y+1)}{2}$$

**Question 1** - Cette fonction, appelée fonction de couplage de Cantor, réalise une numérotation dite *en triangle* des éléments de $\mathbb{N}^2$. Justifier par une figure cette terminologie.

> <img src="num_triangle.png" width=300 height=300 />

**Question 2** - Écrire en OCaml la fonction `cantor : int -> int -> int`.

In [15]:
let cantor x y =
  x + (x+y)*(x+y+1)/2
;;

val cantor : int -> int -> int = <fun>


**Question 3** - Écrire une fonction *récursive* `cantor_rec : int -> int -> int` réalisant cette même numérotation.

In [16]:
let rec cantor_rec x y =
  match x with
  | 0 -> y*(y+1)/2
  | _ -> 1 + cantor_rec (x-1) (y+1)
;;

val cantor_rec : int -> int -> int = <fun>


In [17]:
let rec cantor_rec x y =
  match x, y with
  | 0, 0 -> 0
  | 0, y -> 1 + cantor_rec (y-1) 0
  | _, _ -> 1 + cantor_rec (x-1) (y+1)
;;

val cantor_rec : int -> int -> int = <fun>


**Question 4** - Montrer soigneusement que la fonction `cantor_rec` termine.

<div class="alert alert-success" role="alert">
    
*Pour la première version*
    
Montrons par récurrence sur $x$ que pour tout $x \in \mathbb{N}$, pour tout $y \in \mathbb{N}$, `cantor_rec` $x$ $y$ termine.
    
* Soit $y \in \mathbb{N}$, l'appel `cantor_rec` $0$ $y$ ne nécessite aucun appel récursif et termine.
* Soit $x \in \mathbb{N}^*$, on suppose que pour tout $y \in \mathbb{N}$,  `cantor_rec` $(x-1)$ $y$ termine.
    Alors pour tout $y \in \mathbb{N}$, l'appel `cantor_rec` $x$ $y$ effectue un seul appel récursif `cantor_rec` (x-1) (y+1) qui termine et dont la terminaison assure celle de la fonction.    
</div>

<div class="alert alert-success" role="alert">
    
*Pour la seconde version*
    
On utilise l'ordre lexicographique sur $\mathbb{N}^2$ pour le couple $(x+y,x)$.
    
* Si $(x+y,x)= (0, 0)$ alors $x=y=0$ et `cantor_rec` $x$ $y$ ne nécessite aucun appel récursif et termine.
* Sinon, 
    - ou bien $x = 0$ et `cantor_rec` $x$ $y$ réalise un appel récursif `cantor_rec` $x'$ $y'$ avec $(x'+y',x') = (y-1, y-1)$, qui vérifie $(x'+y',x') \prec (x+y,x)$, et la terminaison de cet appel assure celui de la fonction.
    - ou bien $x > 0$ et `cantor_rec` $x$ $y$ réalise un appel récursif `cantor_rec` $x'$ $y'$ avec $(x'+y',x') = (x+y, x-1)$, qui vérifie $(x'+y',x') \prec (x+y,x)$, et la terminaison de cet appel assure celui de la fonction.
</div>

**Question 5** - Dans cette question, on numérote les éléments de $\mathbb{N}^2$ en carré comme illustré sur la figure ci-dessous ; écrire une fonction récursive `carre : int -> int -> int` réalisant cette numérotation.

<img src="num_carre.png" width=300 height=300 />

In [18]:
let rec carre x y =
  if x = 0
  then y*y
  else
    if y < x
    then 1 + carre x (y+1)
    else 1 + carre (x-1) y
;;

val carre : int -> int -> int = <fun>


In [19]:
let rec carre x y =
  if x = 0
  then 
    if y = 0
    then 0
    else carre (y-1) 0
  else
    if y < x
    then 1 + carre x (y+1)
    else 1 + carre (x-1) y
;;

val carre : int -> int -> int = <fun>


**Question 6** -  Montrer que la fonction `carre` termine.

<div class="alert alert-success" role="alert">
    
*Pour la première version*
    
Montrons par récurrence sur $x$ que pour tout $x \in \mathbb{N}$, pour tout $y \in \mathbb{N}$, `carre` $x$ $y$ termine.
    
* Soit $y \in \mathbb{N}$, l'appel `carre` $0$ $y$ ne nécessite aucun appel récursif et termine.
* Soit $x \in \mathbb{N}^*$, on suppose que pour tout $y \in \mathbb{N}$,  `carre` $(x-1)$ $y$ termine.
    Soit $y \in \mathbb{N}$,
    - Si $y \geq x$, alors on effectue un seul appel récursif `carre` $(x-1)$ $y$ qui termine et dont la terminaison assure celle de la fonction. 
    - Sinon : montrons que pour tout $k \in \{0, ..., x\}$, `carre` $x$ $(x-k)$ termine ;
        * `carre` $x$ $(x-0)$ termine car $x - 0 \geq x$.
        * Soit $k \in \{0, ..., x-1\}$, on suppose que `carre` $x$ $(x-k)$ termine, alors on effectue un unique appel récursif `carre` $x$ $(x-k)$ qui termine.
    
    Par conséquent, si $y < x$, alors pour $k =  x - y$ : l'appel `carre` $x$ $y$ termine 
</div>

<div class="alert alert-success" role="alert">
    
*Pour la seconde version*
    
On utilisera l'ordre lexicographique sur $\mathbb{N}^2$ pour le couple $(x+y,x)$.